In [1]:
import numpy as np
import pandas as pd
import os,sys

In [2]:
cutFlowFile = './HEPData-ins2080541-v1-csv/CutFlow,Chargino.csv'
# cutFlowFile = './HEPData-ins2080541-v1-csv/CutFlow,Stau.csv'
# cutFlowFile = './HEPData-ins2080541-v1-csv/CutFlow,R-hadron.csv'
# cutFlowFile = './HEPData-ins2080541-v1-csv/CutFlow,R-hadron,compressed.csv'

In [3]:
# Get blocks for each parameter point (it is assumed they are separated by the '#: Coupling' tag)
with open(cutFlowFile,'r') as f:
    data = f.read()
    tag = '#: Model'
    blocks = data.split(tag)
    for ib,b in enumerate(blocks):
        blocks[ib] = tag.replace('# :','')+b
        break
# Skip header
blocks = blocks[1:]


In [4]:
dataDict = {}
for b in blocks:
    b = b.split('\n')
    mass = eval(b[0].split('=')[1].split(',')[0].replace('TeV','*TeV').replace('GeV','*GeV'),
                {'TeV' : 1000.0,'GeV' : 1.0})
    tau = eval(b[1].split(',')[1].replace('ns','*ns'),
                {'ns' : 1})
    print(mass,tau)
    dataDict['mLLP'] = [mass]
    dataDict['tau_ns'] = [tau]
    dataType = b[2].split(',')[1]
    for l in b[3:]:
        l = l.strip()
        if not l: continue
        cut,val = l.rsplit(',',1)
        val = eval(val)
        if not cut in dataDict:
            dataDict[cut] = [[0.,0.]]
        if dataType == 'Events':
            dataDict[cut][0][0] = val
        else:
            dataDict[cut][0][1] = val

1300.0 10
1300.0 10


In [5]:
for k,v in dataDict.items():
    if isinstance(v[0],list):
        dataDict[k][0] = tuple(v[0])

In [6]:
# Create dataFrame
df = pd.DataFrame.from_dict(dataDict)
# Fix notation
df.rename(columns={'"$dE/dx\in[1.8,2.4] {MeV~cm^{2}/g}$"' : '(SR-Low - no mass Window)',
                   '$dE/dx>2.4 {MeV~cm^{2}/g}$' : '(SR-High - no mass Window)',
                  '"$m_{T}({track},{p}_{{T}}^{{ miss}}) > 130$ GeV"' : '$m_{T}({track},{p}_{{T}}^{{ miss}}) > 130$ GeV'},
                  inplace=True)

In [7]:
pickleFile = os.path.basename(cutFlowFile).replace(',','_').replace('.csv','.pcl')
df.to_pickle(pickleFile)

In [8]:
df.T

,0
mLLP,1300.0
tau_ns,10
Total,"(20.8, 1.0)"
Event and jet Cleaning,"(20.6, 0.995)"
Trigger,"(11.2, 0.539)"
Primary vertex,"(11.2, 0.539)"
$E_{T}^{miss}>170$ GeV,"(7.9, 0.381)"
Track in PV,"(7.8, 0.378)"
$p_{T} > 50$ GeV,"(6.8, 0.326)"
Track quality requirements,"(5.8, 0.279)"
